# LLM Response Analysis: Cancer Survival Prediction

This notebook analyzes the LLM's predictions and compares them to ground truth cancer outcome data.

**Metrics computed:**
- Confusion Matrix
- Accuracy, Precision, Recall, F1-Score
- AUROC (Area Under ROC Curve)
- AUPRC (Area Under Precision-Recall Curve)
- Classification Report by cancer system

In [1]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score,
    precision_score, 
    recall_score, 
    f1_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score,
    ConfusionMatrixDisplay
)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

## 1. Load Results

In [2]:
import json

# Load the results file
RESULTS_FILE = 'results_rag.json'  # Update path if needed

with open(RESULTS_FILE, 'r') as f:
    results = json.load(f)

print(f"Loaded {len(results)} results")
print(f"\nSample result keys: {list(results[0].keys())}")

Loaded 100 results

Sample result keys: ['prompt', 'response', 'patient_id', 'ground_truth', 'features']


In [ ]:
# Display a sample result
print("Sample prompt (truncated):")
print(results[0]['prompt'][:500], "...")
print("\n" + "="*50)
print("\nSample response:")
print(results[0]['response'])
print("\n" + "="*50)
print("\nGround truth:")
print(json.dumps(results[0]['ground_truth'], indent=2))

## 2. Extract Predictions from LLM Responses

The LLM is asked to return `1` for survival and `0` for death. We need to parse the response to extract the prediction.

In [ ]:
def extract_prediction(response: str) -> int | None:
    """
    Extract the binary prediction (0 or 1) from the LLM response.
    Returns None if no clear prediction is found.
    """
    response = response.strip().lower()
    
    # Try to find explicit 0 or 1 in the response
    # Look for patterns like "0", "1", "prediction: 0", "return 1", etc.
    
    # Check for explicit "0" or "1" as the only content or at the end
    if response == '0' or response == '1':
        return int(response)
    
    # Look for patterns like "prediction: 1" or "answer: 0"
    patterns = [
        r'(?:prediction|answer|result|outcome)[:\s]+([01])\b',
        r'\breturn[s]?\s+(?:a\s+)?([01])\b',
        r'\b(?:predicted|predict)\s+(?:survival\s+)?(?:outcome)?[:\s]*([01])\b',
        r'\b([01])\s*(?:\.|$)',  # 0 or 1 at end of sentence or response
    ]
    
    for pattern in patterns:
        match = re.search(pattern, response)
        if match:
            return int(match.group(1))
    
    # Look for survival/death keywords if no explicit number
    survival_keywords = ['survive', 'survival', 'alive', 'will live', 'likely to survive']
    death_keywords = ['death', 'die', 'dead', 'will not survive', 'unlikely to survive', 'deceased']
    
    for keyword in survival_keywords:
        if keyword in response:
            return 1
    
    for keyword in death_keywords:
        if keyword in response:
            return 0
    
    return None  # Could not determine prediction

# Test the extraction function
test_responses = [
    "1",
    "0",
    "The predicted survival outcome is 1.",
    "Based on the data, I predict 0 (death).",
    "This patient is likely to survive.",
]

print("Testing extraction function:")
for resp in test_responses:
    print(f"  '{resp}' -> {extract_prediction(resp)}")

In [ ]:
# Extract predictions and ground truth
data = []

for item in results:
    prediction = extract_prediction(item['response'])
    
    # Ground truth: died_from_cancer (1 = died from cancer, 0 = alive or died from other cause)
    # Note: We're predicting SURVIVAL (1 = survive, 0 = death)
    # So we need to invert: if died_from_cancer=1, then survival=0
    died_from_cancer = int(item['ground_truth']['died_from_cancer'])
    ground_truth_survival = 1 - died_from_cancer  # Invert for survival prediction
    
    data.append({
        'patient_id': item.get('patient_id'),
        'response': item['response'],
        'prediction': prediction,
        'ground_truth_survival': ground_truth_survival,
        'died_from_cancer': died_from_cancer,
        'is_alive': int(item['ground_truth']['is_alive']),
        'cancer_system': item.get('features', {}).get('cancer_system', 'Unknown'),
        'survival_months': float(item['ground_truth'].get('survival_months', 0)),
    })

df = pd.DataFrame(data)
print(f"Total samples: {len(df)}")
print(f"Samples with valid predictions: {df['prediction'].notna().sum()}")
print(f"Samples with missing predictions: {df['prediction'].isna().sum()}")

In [ ]:
# Show samples where prediction extraction failed
failed_extractions = df[df['prediction'].isna()]
if len(failed_extractions) > 0:
    print(f"\nFailed extractions ({len(failed_extractions)}):")
    for idx, row in failed_extractions.head(5).iterrows():
        print(f"\n  Response: {row['response'][:200]}...")
else:
    print("All predictions extracted successfully!")

In [ ]:
# Filter to valid predictions only
df_valid = df[df['prediction'].notna()].copy()
df_valid['prediction'] = df_valid['prediction'].astype(int)

print(f"\nValid predictions: {len(df_valid)} / {len(df)} ({100*len(df_valid)/len(df):.1f}%)")
print(f"\nPrediction distribution:")
print(df_valid['prediction'].value_counts())
print(f"\nGround truth distribution (survival):")
print(df_valid['ground_truth_survival'].value_counts())

## 3. Confusion Matrix

In [ ]:
y_true = df_valid['ground_truth_survival'].values
y_pred = df_valid['prediction'].values

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Death (0)', 'Survival (1)'])
disp.plot(cmap='Blues', ax=ax, values_format='d')
plt.title('Confusion Matrix: LLM Survival Predictions vs Ground Truth', fontsize=14)
plt.tight_layout()
plt.savefig('confusion_matrix_rag.png', dpi=150, bbox_inches='tight')
plt.show()

# Print raw values
print("\nConfusion Matrix:")
print(f"  TN (True Death):     {cm[0,0]}")
print(f"  FP (False Survival): {cm[0,1]}")
print(f"  FN (False Death):    {cm[1,0]}")
print(f"  TP (True Survival):  {cm[1,1]}")

## 4. Classification Metrics

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Specificity (true negative rate)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Print metrics
print("="*50)
print("CLASSIFICATION METRICS")
print("="*50)
print(f"Accuracy:    {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision:   {precision:.4f}")
print(f"Recall:      {recall:.4f} (Sensitivity)")
print(f"Specificity: {specificity:.4f}")
print(f"F1-Score:    {f1:.4f}")
print("="*50)

In [ ]:
# Full classification report
print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Death (0)', 'Survival (1)']))

## 5. ROC Curve and AUROC

In [ ]:
# Calculate ROC curve (using predictions as scores since they're binary)
# For a proper AUROC, you'd want probability scores, but we'll use binary predictions
try:
    auroc = roc_auc_score(y_true, y_pred)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(fpr, tpr, 'b-', linewidth=2, label=f'LLM (AUROC = {auroc:.4f})')
    ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUROC = 0.5)')
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title('ROC Curve: LLM Survival Predictions', fontsize=14)
    ax.legend(loc='lower right')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1.05])
    plt.tight_layout()
    plt.savefig('roc_curve_rag.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nAUROC: {auroc:.4f}")
except ValueError as e:
    print(f"Could not compute AUROC: {e}")
    print("This may happen if all predictions are the same class.")

## 6. Precision-Recall Curve and AUPRC

In [ ]:
try:
    # Calculate precision-recall curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_true, y_pred)
    auprc = average_precision_score(y_true, y_pred)
    
    # Baseline (proportion of positive class)
    baseline = y_true.mean()
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(recall_curve, precision_curve, 'b-', linewidth=2, label=f'LLM (AUPRC = {auprc:.4f})')
    ax.axhline(y=baseline, color='k', linestyle='--', linewidth=1, label=f'Baseline (AUPRC = {baseline:.4f})')
    ax.set_xlabel('Recall', fontsize=12)
    ax.set_ylabel('Precision', fontsize=12)
    ax.set_title('Precision-Recall Curve: LLM Survival Predictions', fontsize=14)
    ax.legend(loc='lower left')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1.05])
    plt.tight_layout()
    plt.savefig('pr_curve_rag.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nAUPRC: {auprc:.4f}")
    print(f"Baseline (no-skill): {baseline:.4f}")
except ValueError as e:
    print(f"Could not compute AUPRC: {e}")

## 7. Performance by Cancer System

In [ ]:
# Calculate metrics per cancer system
cancer_systems = df_valid['cancer_system'].unique()
system_metrics = []

for system in cancer_systems:
    subset = df_valid[df_valid['cancer_system'] == system]
    if len(subset) < 5:  # Skip systems with too few samples
        continue
    
    y_true_sub = subset['ground_truth_survival'].values
    y_pred_sub = subset['prediction'].values
    
    system_metrics.append({
        'Cancer System': system,
        'N': len(subset),
        'Accuracy': accuracy_score(y_true_sub, y_pred_sub),
        'Precision': precision_score(y_true_sub, y_pred_sub, zero_division=0),
        'Recall': recall_score(y_true_sub, y_pred_sub, zero_division=0),
        'F1': f1_score(y_true_sub, y_pred_sub, zero_division=0),
    })

df_systems = pd.DataFrame(system_metrics).sort_values('Accuracy', ascending=False)
print("Performance by Cancer System:")
print(df_systems.to_string(index=False))

In [ ]:
# Visualize accuracy by cancer system
if len(df_systems) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(df_systems['Cancer System'], df_systems['Accuracy'], color='steelblue')
    ax.axvline(x=accuracy, color='red', linestyle='--', linewidth=2, label=f'Overall: {accuracy:.2f}')
    ax.set_xlabel('Accuracy', fontsize=12)
    ax.set_title('Prediction Accuracy by Cancer System', fontsize=14)
    ax.set_xlim([0, 1])
    ax.legend()
    
    # Add value labels
    for bar, acc in zip(bars, df_systems['Accuracy']):
        ax.text(acc + 0.01, bar.get_y() + bar.get_height()/2, f'{acc:.2f}', va='center')
    
    plt.tight_layout()
    plt.savefig('accuracy_by_system_rag.png', dpi=150, bbox_inches='tight')
    plt.show()

## 8. Error Analysis

In [ ]:
# Analyze errors
df_valid['correct'] = df_valid['prediction'] == df_valid['ground_truth_survival']
df_valid['error_type'] = 'Correct'
df_valid.loc[(df_valid['prediction'] == 1) & (df_valid['ground_truth_survival'] == 0), 'error_type'] = 'False Positive (predicted survival, actually died)'
df_valid.loc[(df_valid['prediction'] == 0) & (df_valid['ground_truth_survival'] == 1), 'error_type'] = 'False Negative (predicted death, actually survived)'

print("Error Type Distribution:")
print(df_valid['error_type'].value_counts())

In [ ]:
# Survival months distribution for correct vs incorrect predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Correct predictions
correct = df_valid[df_valid['correct']]
incorrect = df_valid[~df_valid['correct']]

axes[0].hist(correct['survival_months'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0].set_xlabel('Survival Months')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Correct Predictions (n={len(correct)})')

axes[1].hist(incorrect['survival_months'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1].set_xlabel('Survival Months')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Incorrect Predictions (n={len(incorrect)})')

plt.tight_layout()
plt.savefig('survival_distribution_by_correctness_rag.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nMean survival months (correct):   {correct['survival_months'].mean():.1f}")
print(f"Mean survival months (incorrect): {incorrect['survival_months'].mean():.1f}")

## 9. Summary Dashboard

In [ ]:
# Create summary
print("=" * 60)
print("          LLM SURVIVAL PREDICTION - SUMMARY REPORT")
print("=" * 60)
print(f"\nDataset:")
print(f"  Total samples:           {len(df)}")
print(f"  Valid predictions:       {len(df_valid)} ({100*len(df_valid)/len(df):.1f}%)")
print(f"  Ground truth survival:   {y_true.sum()} ({100*y_true.mean():.1f}%)")
print(f"  Ground truth death:      {len(y_true) - y_true.sum()} ({100*(1-y_true.mean()):.1f}%)")
print(f"\nPrimary Metrics:")
print(f"  Accuracy:                {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  F1-Score:                {f1:.4f}")
try:
    print(f"  AUROC:                   {auroc:.4f}")
except:
    print(f"  AUROC:                   N/A")
try:
    print(f"  AUPRC:                   {auprc:.4f}")
except:
    print(f"  AUPRC:                   N/A")
print(f"\nDetailed Metrics:")
print(f"  Precision:               {precision:.4f}")
print(f"  Recall (Sensitivity):    {recall:.4f}")
print(f"  Specificity:             {specificity:.4f}")
print(f"\nConfusion Matrix:")
print(f"  True Negatives (TN):     {tn}")
print(f"  False Positives (FP):    {fp}")
print(f"  False Negatives (FN):    {fn}")
print(f"  True Positives (TP):     {tp}")
print("=" * 60)

In [ ]:
# Save metrics to JSON for programmatic access
metrics_summary = {
    'total_samples': len(df),
    'valid_predictions': len(df_valid),
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'specificity': float(specificity),
    'f1_score': float(f1),
    'confusion_matrix': {
        'tn': int(tn),
        'fp': int(fp),
        'fn': int(fn),
        'tp': int(tp)
    }
}

try:
    metrics_summary['auroc'] = float(auroc)
except:
    pass

try:
    metrics_summary['auprc'] = float(auprc)
except:
    pass

with open('metrics_summary_rag.json', 'w') as f:
    json.dump(metrics_summary, f, indent=2)

print("Metrics saved to metrics_summary_rag.json")